In [1]:
import os
import itertools
from anytree import RenderTree, LevelOrderIter

In [2]:
class Node:
    def __init__(self,name,parent=None,children=None,depth=None,ancestors=None):
        self.name = name
        self.parent = parent
        self.children = children
        if depth:
            self.depth = depth
        if ancestors:
            self.ancestors = ancestors
        
    def __str__(self):
        return self.name
    
    def __repr__(self):
        return f"Node({self.name})"
    
    def iterator(self):
        """ iterate tree from self in pre-order depth-first search order """
        yield self
        if self.children:
            for child in self.children:             
                for n in child.iterator():
                    yield n
        else:
            return
        
    def findDepth(self):
        if hasattr(self,"depth"):
            return self.depth
        depth = self.parent.findDepth()+1
        self.depth = depth
        
        return depth
    
    def ancestry(self):
        if hasattr(self,"ancestors"):
            return self.ancestors
        ancestors = self.parent.ancestry()+[self.parent.name]
        self.ancestors = ancestors
        
        return ancestors

In [3]:
def nodeFromName(name,tree_names,tree_nodes):
    for c_name,node in zip(tree_names,tree_nodes):
        if name==c_name:
                return node
    return None

def addChild(parent,child):
    if parent.children:
        parent.children.append(child)
    else:
        parent.children = [child]
    return parent

In [4]:
def createTree(orbits):
    root = Node(orbits[0][0])
    roots = {root}
    for i,[center, satellite] in enumerate(orbits):
        tree_names = [n.name for r in roots for n in r.iterator()]
        tree_nodes = [n for r in roots for n in r.iterator()]
        if center in tree_names:
            parent = nodeFromName(center,tree_names,tree_nodes)
        else:
            parent = Node(center)
            roots.add(parent)
        if satellite in tree_names:
            sat = nodeFromName(satellite,tree_names,tree_nodes)
            sat.parent = parent
        else:
            sat = Node(satellite,parent=parent)
            
        addChild(parent,sat)

        while root.parent:
            root = root.parent

        roots.add(root)
        if sat in roots:
            roots.remove(sat)
            
    root.depth=0
    root.ancestors=[]
    return root

In [5]:
filename = os.path.join(os.getcwd(),"d6_input")
with open(filename) as f:
    orbits=[orbit.strip().split(")") for orbit in f.readlines()]

In [6]:
# import random
# str_input = "COM)B,B)C,C)D,D)E,E)F,B)G,G)H,D)I,E)J,J)K,K)L,K)YOU,I)SAN".split(",") #,K)YOU,I)SAN
# random.shuffle(str_input)
# orbits = [orbit.split(")") for orbit in str_input]
# print(orbits)

In [7]:
root = createTree(orbits)
# print(RenderTree(root))

In [8]:
def orbitsCount(root):
    orbits_sum = 0
    for n in root.iterator():
        orbits_sum+=n.findDepth()
    return orbits_sum

orbitsCount(root)

312697

In [9]:
anc_list =[]
for n in root.iterator():
    if n.name=="SAN" or n.name=="YOU":
        anc_list.append(n.ancestry())

i=0
while anc_list[0][i]== anc_list[1][i]:
    i+=1
len(anc_list[0])+len(anc_list[1])-2*i

466

# Day 7: Amplification Circuit

In [10]:
def paraMode(mode,parameter,intcode_program):
    if mode =="0":
        return intcode_program[parameter]
    elif mode=="1":
        return parameter
    else:
        raise ValueError(f"Unknown mode for parameter {parameter}")
        
def run_program(intcode,auto_mode=False,inputs=None):
    ptr = 0
    skip = (4,4,2,2,3,3,4,4)
    while intcode[ptr]!=99:
        str_opcode = '%05d'%(intcode[ptr])
        mode = str_opcode[:-2][::-1]

        if str_opcode[-1]=='1':
            param = intcode[ptr+1:ptr+4]
            intcode[param[2]]=paraMode(mode[0],param[0],intcode) + paraMode(mode[1],param[1],intcode)

        elif str_opcode[-1]=='2':
            param = intcode[ptr+1:ptr+4]
            intcode[param[2]]= paraMode(mode[0],param[0],intcode) * paraMode(mode[1],param[1],intcode)

        elif str_opcode[-1]=='3':
            if auto_mode:
                input_val=inputs[0]
                inputs.pop(0)
            else:
                input_val = input("Please write the input value:")
            intcode[intcode[ptr+1]]=int(input_val)

        elif str_opcode[-1]=='4': 
            if auto_mode:
                yield paraMode(mode[0],intcode[ptr+1],intcode)
            else:
                print(paraMode(mode[0],intcode[ptr+1],intcode))
        elif str_opcode[-1]=='5':
            param = intcode[ptr+1:ptr+3]
            true = paraMode(mode[0],param[0],intcode)
            ptr = (paraMode(mode[1],param[1],intcode)-3)*(true>0)+ptr*(true<=0)

        elif str_opcode[-1]=='6':
            param = intcode[ptr+1:ptr+3]
            false = paraMode(mode[0],param[0],intcode)
            ptr = (paraMode(mode[1],param[1],intcode)-3)*(false<=0)+ptr*(false>0)
            
        elif str_opcode[-1]=='7':
            param = intcode[ptr+1:ptr+4]
            less_than=paraMode(mode[0],param[0],intcode)<paraMode(mode[1],param[1],intcode)
            intcode[intcode[ptr+3]]=1*less_than

        elif str_opcode[-1]=='8':
            param = intcode[ptr+1:ptr+4]
            equal = (paraMode(mode[0],param[0],intcode)==paraMode(mode[1],param[1],intcode))
            intcode[intcode[ptr+3]]=1*equal

        else:
            raise ValueError(f"Something went wrong. Opcode is {intcode[i]} at index {i} for the current program {intcode}")
        
        ptr += skip[int(str_opcode[-1])-1]
        
    yield None

In [11]:
filename = os.path.join(os.getcwd(),"d7_input")
with open(filename) as f:
    initial_program = [int(opcode) for opcode in f.read().split(",")]    
    
# initial_program = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
# initial_program = [3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0]
# initial_program = [3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0]

In [19]:
# Q1
def maxThrusterSignal(initial_program):
    max_thruster_signal=0
    for seq in list(itertools.permutations(range(5))):
        inp = 0
        for phase in seq:
            inputs = [phase,inp]
            inp = next(run_program(initial_program[:],auto_mode=True,inputs=inputs))
        max_thruster_signal=max(max_thruster_signal,inp)
    return max_thruster_signal
        
max_thruster_signal = maxThrusterSignal(initial_program)
max_thruster_signal

844468

In [ ]:
# Q2

# initial_program = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]
# seq = [9,8,7,6,5]

# initial_program = [3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,
# -5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,
# 53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10]
# seq= [9,7,8,5,6]

In [18]:
def maxThrusterSignalFeedback(initial_program):
    max_signal = 0
    for seq in list(itertools.permutations(range(5,10))):
        inputs= [[phase] for phase in seq]
        inputs[0].append(0)
        amps = [run_program(initial_program[:],auto_mode=True,inputs=inp) for inp in inputs]

        cur_amp=0
        out = -1
        while out is not None:
            last_out = out
            out = next(amps[cur_amp])
            if out is None:
                break
            inputs[(cur_amp+1)%5].append(out)
            cur_amp = (cur_amp+1)%5
        max_signal=max(max_signal,last_out)
    return max_signal

max_signal = maxThrusterSignalFeedback(initial_program)
max_signal

4215746